# Python Data Preparation Workshop
September 2019

* ## 1. (Nominal) Categorical Variables -- Distinct values but not *ordered*
  * You will convert categorical variables into usable predictive variables.
* ## 2. Imputation -- Filling in Missing Variables
  * You will see how to fill in missing variables with the Titanic dataset
  * Optionally, you will be able to run CatBoost for data which has not had significant missing and categorical pre-processing.
* ## 3. Cyclical Variables -- Interesting challenges for models which assume ordered, continuous variables
  * You will have an opportunity to model a cyclical variable as two continuous variables.
   
Note: a companion workbook on data preparation techniques discusses concepts in greater detail.  This workbook is for a workshop to gain hands-on experience with related python code.

Expectation is some familiarity with programming or pandas/python.  *Feel free to work with a partner if you are unfamiliar with python and jupyter.*  There are a couple places where pandas syntax may be required and the comments are intended to guide you through this.

Anaconda installation: https://docs.google.com/document/d/11N6IAy3NeD6eMv5jxq5h9jOW_dVTfTs8txHsELzCx9w/edit?usp=sharing
There is a small section using CatBoost, and installation instructions for it are at the end of the Anaconda document above.  You may decide to skip the CatBoost tutorial section if you have difficulty with installation.

All sections are to be "run" in sequence.  Additional instructions are provided when work is required.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge, LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score
import sklearn
import catboost
print('Checking installed versions:')
print(f'pandas version installed {pd.__version__} -- should be at least 0.24.2')
print(f'sklearn version installed {sklearn.__version__} -- should be at least 0.21.2')
print(f'catboost version installed {catboost.__version__} -- should be at least 0.17.1')

## Titanic Data Set 

On April 14, 1912 the Titanic hit an iceberg and it sank on April 15.  An estimated 710 passengers were saved and 1,514 died.  A dataset has been derived from this disaster (data circa 1999) and used by universities and adopted by Kaggle.

* Dataset is in the data/titanic subdirectory of this project
* Observations are per passenger.  PassengerId exists but will not be used as it is arbitrary.

**survival**:	Did passenger survive the Titanic crash? (0 = No, 1 = Yes)

**pclass**:	Ticket class (1 = 1st, 2 = 2nd, 3 = 3rd)

**sex**:	Sex of passenger

**Age**:	Age in years

**sibsp**:	# of siblings / spouses aboard the Titanic (see note)

**parch**:	# of parents / children aboard the Titanic (see note)

**ticket**:	Ticket number

**fare**:	Passenger fare (GBP)

**cabin**:	Cabin number

**embarked**:	Port of Embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)

More details about the data set are found at: http://campus.lakeforest.edu/frank/FILES/MLFfiles/Bio150/Titanic/TitanicMETA.pdf.  From that note:

"With respect to the family relation variables (i.e. sibsp and parch) some relations were
ignored. The following are the definitions used for **sibsp** and **parch**.
Sibling: Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic
Spouse: Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiances
Ignored)
Parent: Mother or Father of Passenger Aboard Titanic
Child: Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic
Other family relatives excluded from this study include cousins, nephews/nieces,
aunts/uncles, and in-laws. Some children travelled only with a nanny, therefore parch=0
for them. As well, some travelled with very close friends or neighbors in a village,
however, the definitions do not support such relations."


In [ ]:
# Loading titanic from this repo.  The data set has been split into train and test; we will work with train here.
train_df = pd.read_csv('data/titanic/train.csv')
train_df.info()
# Note that there are 891 observations for PassengerId.
# Columns with Nulls will report fewer than 891 observations.  These have to be handled as missing data

# Note: titanic is also in the catboost repo, but not used in case a student has not installed catboost.

### Dependent Variable: Whether passenger survived the Titanic crash

In [ ]:
# Use value_counts() to get distribution:
train_df.Survived.value_counts()

# Notice how Survived has 891 non-null values, and they are 0 and 1
# Remember this frequency distribution syntax for later.

## Imputation and Data Processing -- Handle Missing Values: Fewer than 891 observations

* Embarked
* Age
* Cabin

### Embarked: What do we know about the two passengers not listed as embarked?

In [ ]:
# Pandas syntax for checking NULLs.  Notice how pandas uses something like a SQL where clause in square brackets.
train_df[train_df.Embarked.isnull()==True]

### What action should we take for observations without an embarkation port?

* Embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)

In [ ]:
# Get the count of passengers without a Null Embarked
train_df[train_df.Embarked.isnull()==True].PassengerId.count()


## # Use pandas.get_dummies() to one hot encode Embarked ports

In [ ]:
# Calls get_dummies and concatenates this to the data set
train_df = pd.concat([train_df, pd.get_dummies(train_df.Embarked, prefix='Embarked')], axis=1)

# note new columns added to dataframe
train_df.info()

# note how new columns relate to the Embarked column
sample_table = train_df[train_df.Embarked.isnull()==False].head(5)
display(HTML(sample_table.to_html()))  # fancy syntax for pretty printing tables in jupyter

### Age

In [ ]:
train_df.Age.describe()

In [ ]:
train_df.Age.hist(figsize=(12,6))

### Use SimpleImputer to create a sample column -- illustrating what happens if we impute age as mean when missing

* This is just to demonstrate how the code works, we will use a more sophisticated imputation of Age below
* One could trivially do these steps without the class "SimpleImputer" but it could prove useful at scale (multiple variables)


In [ ]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(train_df[['Age']])
print(imp_mean.transform(train_df[['Age']])[:20])
# Note how all imputed values are about 30 years of age.  Feel free to try another strategy such as median or most_frequent

### Age: Let's impute it from other variables rather than simply using mean

* We will first convert other object variables to features
* Then we will impute Age once other variables are ready

<HR>

## Cabin

* Cabin does not look that useful; maybe the first character means something

In [ ]:
# Exercise: Write code to show how many records are missing Cabin.  (See above for example)






In [ ]:
# Exercise: Look at the some sample values for Cabin 






In [ ]:
# Seems like the first character in Cabin is the cabin deck.  
# Here you create a new variable based on the first character
train_df['CabinDeck'] = train_df.Cabin.str.slice(0,1)

In [ ]:
# Exercise: Print out the frequency distribution for CabinDeck (see above for examples)






In [ ]:
# Verify that CabinDeck is missing as often as Cabin
train_df[train_df.CabinDeck.isnull()==True].PassengerId.count()

### Thought Experiment: What do you want to do about Cabins?

* We have Cabins for less than half of the variables?
* Even when we do have cabins, the T level and G level have few observations, and the T level is perhaps wrong 
(since it would be the Tank Top)

In [ ]:
# examine sample data
train_df[train_df.CabinDeck=='T']

In [ ]:
# examine sample data
train_df[train_df.CabinDeck=='G']

Thought Experiments

* Do you want to drop the column Cabin because so much information is missing?
* Would you impute the column?
* Or we could use the CabinDeck and keep a category for "no information", which might tell us something about the class or person

In [ ]:
train_df = pd.concat([train_df, pd.get_dummies(train_df.CabinDeck, prefix='CabinDeck')], axis=1)

# resulting columns:
print(train_df.info())

# resulting data:
sample_table = train_df[train_df.CabinDeck.isnull()==False].head(5)
display(HTML(sample_table.to_html()))
# notice how dummy columns correspond to the values in CabinDeck

In [ ]:
## Remove the CabinDeck_T variable since T does not seem like a proper cabin (by removing it we treat it similar to passengers without cabin deck)
train_df.drop(columns=['CabinDeck_T'], inplace=True)
train_df.info()

## Categorical Variables - What Other Variables Should be Considered?

* Variables type object are all strings.  What will you do with them?

In [ ]:
train_df.dtypes[train_df.dtypes=='object']

### Sex

In [ ]:
train_df.Sex.value_counts()

### Approach #1: Pandas get_dummies - one hot encode Sex

In [ ]:
# Exercise: Use pandas get_dummies to create new variables (see above example)
# Verify that the values appear sensible.





### Approach #2: scikit-learn one hot encoder -- illustrating another approach

* This approach demonstrates other methods to accomplish the same objective
* We will print out results but not use this method

When would you use these two approaches?
* pd.get_dummies - quick and useful in pandas
* OneHotEncoder - use when in numpy and not in pandas, simply using arrays not dataframes

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(train_df.Sex.to_frame())
print(enc.categories_)
transform_example = enc.transform(train_df.Sex.to_frame())
# Note: with one hot encoding you will have an array similar to above, but you have to do the work to put it back in a dataframe
transform_example[:10].toarray()

### Ticket

Does this look useful to you?

In [ ]:
train_df.Ticket.value_counts().head(10)

## Review other quantitative variales

In [ ]:
# Exercise: describe Fare, and determine whether the data appears sensible
# 
# Thought experiment -- Do you wish to modify any of the Fares?





Pclass and Parch variables

In [ ]:
train_df.Pclass.value_counts()

In [ ]:
# number of parents/children aboard ship
train_df.Parch.value_counts()

### Does one have questions about accuracy of this data?  7 people had 8 spouses / siblings aboard ship?

In [ ]:
# One way to explore the data might be to look at the person who had 6 children
train_df[train_df.Parch==6]

In [ ]:
# number of siblings or spouses aboard ship.
train_df.SibSp.value_counts()

In [ ]:
# Exercise: Investigate passengers with 5 or 8 Siblings or Spouses. Is this data credible?
#
# if behind, please skip to next step...
#





## Milestone Check - Where Are We?

* Plan to exclude dependent variable:
  * Survived

* Do not plan to use:
  * PassengerId    
  * Ticket
  * Name -- not investigated yet
  
* Features created from raw variables:
  * male/female: Replace Sex.  In some instances use one
  * CabinDeck: instead of Cabin
  * Embarked_S, Embarked_Q, Embarked_C: rather than Embarked
  
* Numeric raw variables available:
  * SibSp
  * Fare
  * Pclass
  * Parch
  
* Now we plan to impute **Age** which has some missing values


## Return to Imputation 
### Create a new column called AgeImputedPred: Predict the age from all other variables

In [ ]:
# Examine some sample data with Age missing.
train_df[train_df.Age.isnull()==True].head(6)
# ... especially interesting that titles like "Mr.", "Mrs.", "Miss", "Master" might suggest age

In [ ]:
# Syntax to pick out a few titles
display(HTML(train_df[train_df.Name.str.contains('Miss')].head(3).to_html()))
display(HTML(train_df[train_df.Name.str.contains('Master')].head(3).to_html()))
display(HTML(train_df[train_df.Name.str.contains('Mr.')].head(3).to_html()))

In [ ]:
# Could we use title of name for averaging of ages?
known_titles = ['Mr.', 'Mrs.', 'Miss', 'Master', 'Rev.', 'Col.', 'Mlle.', 'Dr.', 'Ms.', 'Major', 'Mme.', 'Don.', 'Capt.', 'Jonkheer', 'Countess']

for title in known_titles:
    has_title = train_df[train_df.Name.str.contains(title)]
    print(f'{title}: Count - {has_title.Age.count()}; Mean Age - {np.round(has_title.Age.mean(),1)}')
    missing_age = has_title[has_title.Age.isnull()==True]
    print(f'  # missing age is: {missing_age.PassengerId.count()}')
print(train_df[train_df.Name.str.contains('|'.join(known_titles))==False].Name)

### Optional Steps: For Students With Lots of Time...

* It would be helpful to create a categorical variable for each of the titles in the data set.
* Then as you go through the code below which imputes age, add in these category variables as well

In [ ]:
# Space for optional exercise for advanced staudents
#
# skip if you do not have time
#








In [ ]:
### Create a new column called AgeImputedPred: Predict the age from all other variables
#
col_list = ['Age', 'Sex_male', 'Sex_female', 'Embarked_S', 'Embarked_Q', 'Embarked_C', 'SibSp', 'Fare', 'Pclass', 'Parch', 'CabinDeck_A', 'CabinDeck_B', 'CabinDeck_C', 'CabinDeck_D', 
            'CabinDeck_E', 'CabinDeck_F', 'CabinDeck_G']
print(train_df[col_list].head(3))
# after cleaning up and having cat variables edit and do this:
iterative_imp = IterativeImputer(random_state=0, estimator=BayesianRidge())
iterative_imp.fit(train_df[col_list])
result = iterative_imp.transform(train_df[col_list])
print(type(result))
print(result[:6, 0])
train_df['Age_Imputed'] = result[: ,0]

In [ ]:
train_df.Age_Imputed.describe()
# Notice anything strange in the imputed ages?

### Charts of Ages: How do Imputed Ages Compare with Acutal Ages?

In [ ]:
ax = train_df[train_df.Age.isnull()==False].Age_Imputed.hist(figsize=(14,6))
plt.suptitle('Original Values of Age')

In [ ]:
ax = train_df[train_df.Age.isnull()==True].Age_Imputed.hist(figsize=(14,6))
plt.suptitle('Imputed Values of Age')

In [ ]:
train_df.Age_Imputed.hist(figsize=(14,6))
plt.suptitle('Passenger Age (Imputed and Non-Imputed)')

In [ ]:
## One might be concerned with ages below zero.  They seem to be a set of children from the same large family.
train_df[train_df.Age_Imputed<0]

In [ ]:
# Optional Exercise: Clean up the negative ages
#
# This assumes pandas experience with loc[].
#
#
# One could substitute a slightly better age, perhaps 1, for these children.  Let's assume they are septuplets!?
#
# Using loc() function set the imputed age to 1.  Modify this line filling in the loc parameters:
train_df.loc[<>, <>] = 1



# Verify no negative ages left
train_df[train_df.Age_Imputed<0]


In [ ]:
# Exercise: again plot the Age_Imputed histogram (.hist function) after making the correction above




## Explore Different Classification Models with these Variables

* What do you notice about model performance?

In [ ]:
y = train_df[['Survived']]
all_cols = set(train_df.columns.tolist())
# the original categorical columns are marked unusable here, so the poss_cols includes new columns and quantitative columns
unusable_cols = set(['PassengerId', 'Survived', 'Name', 'Sex', 'Age', 'Cabin', 'Embarked', 'female', 'CabinDeck', 'Ticket'])
poss_cols = all_cols - unusable_cols
poss_cols

### Model 1: Passenger class and gender

* LogisticRegression returns mean accuracy score

In [ ]:
# Limited model: passenger class and gender only
var = ['Pclass', 'Sex_male']
X = train_df[var]
model = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X, np.ravel(y))
scores = cross_val_score(model, X, np.ravel(y), cv=3)
print(f'Accuracy scores {scores}')


### Model 2: All prepared variables

In [ ]:
# Adding in all columns only slightly improves model over gender and class
X = train_df[poss_cols]
model = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr', max_iter=5000).fit(X, np.ravel(y))
scores = cross_val_score(model, X, np.ravel(y), cv=3)
print(f'Accuracy scores {scores}')


## Model 3 - CatBoost - Optional Section

* If you run out of time to install CatBoost for this tutorial you could skip this section.
* This algorithm will take 3 minutes to execute with modest computing power
    

In [ ]:
!jupyter nbextension enable --py widgetsnbextension
from catboost import Pool, CatBoostClassifier, cv
# must have catboost instaled to run this section

cat_train_df = pd.read_csv('data/titanic/train.csv')
cat_y = cat_train_df[['Survived']]
print(cat_train_df.columns)
cat_train_df.drop(columns=['PassengerId', 'Survived'], inplace=True)
cat_x = cat_train_df
cate_features_index = np.where(cat_x.dtypes != float)[0]
cat_train_df.fillna(-999,inplace=True)
xtrain, xtest, ytrain, ytest = train_test_split(cat_x, cat_y, train_size=.85, random_state=1234)
model = CatBoostClassifier(custom_loss=['Accuracy'],random_seed=42,logging_level='Silent', iterations=1200)

### This next step takes sometime to run, and metric_period determines how often it will output.  The * to the left of the cell will remain while running.

* The plot only seems to work in jupyter notebook not jupyter lab; possibly because the widgetsnbextension is not turned on in lab.

In [ ]:
# Click on the "Accuracy" rather than "LogLoss" to get a sense of accuracy
model.fit(xtrain, ytrain,cat_features=cate_features_index,eval_set=(xtest, ytest), plot=True,  metric_period=10)

In [ ]:
# alternative code with cross validation - advanced users could uncomment and work with this
#
# model = CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42)
# cv_data = cv(Pool(data=cat_x, label=cat_y, cat_features=cate_features_index), model.get_params(), fold_count=3, metric_period=50, plot=True)

<HR>

## Cyclical Variables - How Can Data Preparation Help with Cyclical Variables?

A one dimensional variable such as day of year captures continuous information (progress of human history) as well as cyclic information (seasons or portion of the year).  In this example we show how taking the *sin and cos of the fraction of the cycle* converts the period of the year into two dimensions, allowing similarity to be created for adjacent and seasonal values.

First, an example of code and plotting which you can use below

In [ ]:
# sample code and plotting
np.random.seed(42)
rand_days = np.append(np.random.randint(0, 365, 25), [11, 254, 284])
# print(rand_days)
df=pd.DataFrame(rand_days, columns=['day_num'])
df['sin_of_date'] = np.sin(2*np.pi*df.day_num/365)
df['cos_of_date'] = np.cos(2*np.pi*df.day_num/365)
plt = df.plot.scatter('sin_of_date','cos_of_date', figsize=(10, 10))
plt.set_title('Day of Year Distributed Across Two Variables')
for dname, dnum in (['Jan 11', 11], ['Sep 11', 254], ['Oct 11', 284]):
    _ = plt.text(df[df.day_num==dnum].sin_of_date.values[0]+.02, df[df.day_num==dnum].cos_of_date.values[0], dname)

_ = plt.text (-1, 1, 'Temperature related to cos(date)')
_ = plt.text (-1, -1, 'Business seasons to sin(date)')
print('Similar day next year will have same values on these two features.')
print('You may also elect to have day in human history as a third variable.')

### Workshop Activities:

* Simulate Cyclic Variable
* Perform statistical analysis several ways
  * Treating variable as dummy
  * Treating variable as ordinal
  * Treating variable as cyclic

In [ ]:
# Simulate events occurring at various hours of the day

# Independent variables are time
time_array = np.random.randint(0, 24, size=(2500))

# data checks
print(time_array)
print(min(time_array))
print(max(time_array))
sim_df = pd.DataFrame(time_array, columns=['time_of_day'])

In [ ]:
# Dependent variable:
# Simulate function where dependent variable is driven by absolute difference in time from 10pm (22) plus a Gaussian normal as a stochastic factor
sim_df['hours_before_22'] = np.maximum(0, 22 - sim_df.time_of_day)
sim_df['hours_after_22'] = 24 - 22 + sim_df.time_of_day
sim_df['distance_from_22'] = sim_df[['hours_before_22', 'hours_after_22']].min(axis=1)
display(HTML(sim_df.head(4).to_html()))
np.random.seed(seed=42)
sim_df['y'] = 20 + np.random.normal(size=len(sim_df))*4 - sim_df.distance_from_22
display(HTML(sim_df.head(4).to_html()))
sim_df.plot.scatter(x='distance_from_22', y='y', figsize=(12,8))


### Exercise A: Regress on ordinal value

* Naive model: we do not know underlying algorithm and our only real independent variable is time of day
* print out r^2 scores

Simpified code uses cross_val_score method, which performs k-fold cross validation, seemingly equivalent to larger code block:
kfold = KFold(n_splits=3, shuffle=True, random_state=42)
for i, (train, test) in enumerate(kfold.split(X, y)):
    model.fit(X.iloc[train,:], y.iloc[train,:])
    score = model.score(X.iloc[test,:], y.iloc[test,:])
    scores.append(score)


In [ ]:
# Define dataframes for X and y:
y = sim_df[['y']]
X = sim_df[['time_of_day']] 


In [ ]:
# Naive model: treat time_of_day as an interval variable:
model = LinearRegression()
scores = cross_val_score(model, X, y, cv=3)
print(scores)
# r^2 score is not that great, below 0.1

### Exercise B: Treat hour of day as categorical

* print out r^2 scores

Note: in practice this option would discard information, for example we could not realistically add a categorical variable for every minute.  We are showing it here so you have a benchmark of a model which had complete information with a traditional approach.

In [ ]:
# add_var are hour of day as dummy variables
add_var = pd.get_dummies(X.time_of_day, prefix='hour', drop_first=True)
# Add all the columns to the model data
X = X.join(add_var)
# Drop the original column that was expanded
X.drop(columns=['time_of_day'], inplace=True)
print(X.head(3))

model = LinearRegression()
scores = cross_val_score(model, X, y, cv=3)
print(scores)

# note how the r^2 improved considerably

### Exercise C: Treat hour of day as cyclic variable

* print out r^2 scores

In [ ]:
# Exercise: Define sin_hour and cos_hour per model above
sim_df['sin_hour'] = <>
sim_df['cos_hour'] = <>
X = sim_df[['sin_hour', 'cos_hour']]
model = LinearRegression()
scores = cross_val_score(model, X, y, cv=3)
print(scores)

# comparable results without adding a dummy variable for every time period, something which would be infeasible if hours and minutes were added

In [ ]:
# Advanced users: can you rewrite this simulation using time of day instead of hour of day?  
# What results do you find?




